In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /Users/sunao-mac/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py


In [3]:
t0 = time.time()

## Preparation for Fisher
Computation of $C(l)$ takes ~20 sec for each cosmology.
So this notebook pre-compute $C(l)$ for cosmologies needed for Fisher analysis.

Model parameters of 3x2pt analysis with single(double) source redshift bin are
- comsology: 5
- galaxy bias: 1x3
- magnification bias: 1x3
- photo-z: 1x1(2)
- multiplicative bais: 1x1(2)

13(15) parameters in total.

$C(l)$s depend on cosmological parameters, galaxy bias and magnification bias, for which we need to compute $C(l)$.

**Note** : Fisher analysis is doable even if the computational time for one model reaches ~20 sec. However, in the real analysis, ~20 sec is too much to run sampling code. 
I just want to know how people in cosmic shear, like Hikage-san and Hamana-san, or people in 3x2pt, like DES or KiDS, implement code of $C(l)$.?

In [4]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [5]:
Omega_s_HSC = 400

### List of models to compute $C(l)$ to compute **single** source bin anlaysis
0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph}$ +
13. $\Delta m$ + = easy. Just multiply $1+\Delta m$ to fiducial.

In [6]:
dirname = 'single_source_ClY3'

In [7]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=True)
pk2cl.dump_Cl_cache(f'{dirname}/fiducial', overwrite=True)

:22.07122302055359 sec
saving l to single_source_ClY3/fiducial/l.txt
saving lowz,lowz to single_source_ClY3/fiducial/lowz,lowz.txt
saving lowz,cmass1 to single_source_ClY3/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to single_source_ClY3/fiducial/lowz,cmass2.txt
saving cmass1,lowz to single_source_ClY3/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to single_source_ClY3/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to single_source_ClY3/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to single_source_ClY3/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/fiducial/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/fiducial/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/fiducial/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/fiducial/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/fiducial/s12,s12.txt
saving cosmo_dict to single_source_ClY3/fiducial/cosmo_dict.json.
saved galaxy 

In [8]:
cosmo_fiducial = pk2cl.get_cosmo_dict()
dp = 0.0001

In [9]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_b'] = cosmo_fiducial['omega_b']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_b', overwrite=True)

:12.118392705917358 sec
saving l to single_source_ClY3/omega_b/l.txt
saving lowz,lowz to single_source_ClY3/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/omega_b/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/omega_b/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/omega_b/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/omega_b/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/omega_b/s12,s12.txt
saving cosmo_dict to single_source_ClY3/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 12.9 s, sys: 102 ms, total: 13 s
Wall time: 12.4 s


In [10]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_c'] = cosmo_fiducial['omega_c']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_c', overwrite=True)

:12.6399507522583 sec
saving l to single_source_ClY3/omega_c/l.txt
saving lowz,lowz to single_source_ClY3/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/omega_c/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/omega_c/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/omega_c/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/omega_c/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/omega_c/s12,s12.txt
saving cosmo_dict to single_source_ClY3/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.3 s, sys: 128 ms, total: 13.5 s
Wall time: 12.9 s


In [11]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['Omega_de'] = cosmo_fiducial['Omega_de']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/Omega_de', overwrite=True)

:13.067876815795898 sec
saving l to single_source_ClY3/Omega_de/l.txt
saving lowz,lowz to single_source_ClY3/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/Omega_de/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/Omega_de/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/Omega_de/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/Omega_de/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/Omega_de/s12,s12.txt
saving cosmo_dict to single_source_ClY3/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.7 s, sys: 151 ms, total: 13.8 s
Wall time: 13.4 s


In [12]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['n_s'] = cosmo_fiducial['n_s']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/n_s', overwrite=True)

:12.989488124847412 sec
saving l to single_source_ClY3/n_s/l.txt
saving lowz,lowz to single_source_ClY3/n_s/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/n_s/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/n_s/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/n_s/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/n_s/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/n_s/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/n_s/s12,s12.txt
saving cosmo_dict to single_source_ClY3/n_s/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.6 s, sys: 156 ms, total: 13.8 s
Wall time: 13.3 s


In [13]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['ln10p10As'] = cosmo_fiducial['ln10p10As']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/ln10p10As', overwrite=True)

:12.93839406967163 sec
saving l to single_source_ClY3/ln10p10As/l.txt
saving lowz,lowz to single_source_ClY3/ln10p10As/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/ln10p10As/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/ln10p10As/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/ln10p10As/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/ln10p10As/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/ln10p10As/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/ln10p10As/s12,s12.txt
saving cosmo_dict to single_source_ClY3/ln10p10As/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.6 s, sys: 152 ms, total: 13.7 s
Wall time: 13.3 s


In [14]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78*(1.0+dp), '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1lowz', overwrite=True)

:13.595612049102783 sec
saving l to single_source_ClY3/b1lowz/l.txt
saving lowz,lowz to single_source_ClY3/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/b1lowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/b1lowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/b1lowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/b1lowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/b1lowz/s12,s12.txt
saving cosmo_dict to single_source_ClY3/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 14.2 s, sys: 167 ms, total: 14.3 s
Wall time: 13.9 s


In [15]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10*(1+dp), '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass1', overwrite=True)

:13.19619083404541 sec
saving l to single_source_ClY3/b1cmass1/l.txt
saving lowz,lowz to single_source_ClY3/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/b1cmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/b1cmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/b1cmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/b1cmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/b1cmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY3/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.8 s, sys: 155 ms, total: 14 s
Wall time: 13.5 s


In [16]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28*(1+dp), '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass2', overwrite=True)

:12.826854228973389 sec
saving l to single_source_ClY3/b1cmass2/l.txt
saving lowz,lowz to single_source_ClY3/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/b1cmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/b1cmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/b1cmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/b1cmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/b1cmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY3/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.5 s, sys: 133 ms, total: 13.6 s
Wall time: 13.1 s


In [17]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259*(1.0+dp)])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamaglowz', overwrite=True)

:12.615675926208496 sec
saving l to single_source_ClY3/alphamaglowz/l.txt
saving lowz,lowz to single_source_ClY3/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/alphamaglowz/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/alphamaglowz/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/alphamaglowz/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/alphamaglowz/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/alphamaglowz/s12,s12.txt
saving cosmo_dict to single_source_ClY3/alphamaglowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.3 s, sys: 133 ms, total: 13.4 s
Wall time: 12.9 s


In [18]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563*(1+dp)])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass1', overwrite=True)

:12.922698020935059 sec
saving l to single_source_ClY3/alphamagcmass1/l.txt
saving lowz,lowz to single_source_ClY3/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/alphamagcmass1/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/alphamagcmass1/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/alphamagcmass1/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/alphamagcmass1/s12,s12.txt
saving cosmo_dict to single_source_ClY3/alphamagcmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.5 s, sys: 145 ms, total: 13.7 s
Wall time: 13.2 s


In [19]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729*(1+dp)])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass2', overwrite=True)

:12.88152289390564 sec
saving l to single_source_ClY3/alphamagcmass2/l.txt
saving lowz,lowz to single_source_ClY3/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/alphamagcmass2/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/alphamagcmass2/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/alphamagcmass2/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/alphamagcmass2/s12,s12.txt
saving cosmo_dict to single_source_ClY3/alphamagcmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.5 s, sys: 148 ms, total: 13.6 s
Wall time: 13.2 s


In [20]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', dp, 0.0, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph', overwrite=True)

:12.415724992752075 sec
saving l to single_source_ClY3/dzph/l.txt
saving lowz,lowz to single_source_ClY3/dzph/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/dzph/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/dzph/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/dzph/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/dzph/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/dzph/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/dzph/s12,s12.txt
saving cosmo_dict to single_source_ClY3/dzph/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.1 s, sys: 115 ms, total: 13.2 s
Wall time: 12.7 s


In [21]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s12', 'sourcePzs_Y1/MLZs12.txt', 0.0, dp, 0.2, '8.74 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm', overwrite=True)

:12.824283838272095 sec
saving l to single_source_ClY3/dm/l.txt
saving lowz,lowz to single_source_ClY3/dm/lowz,lowz.txt
saving cmass1,cmass1 to single_source_ClY3/dm/cmass1,cmass1.txt
saving cmass2,cmass2 to single_source_ClY3/dm/cmass2,cmass2.txt
saving lowz,s12 to single_source_ClY3/dm/lowz,s12.txt
saving cmass1,s12 to single_source_ClY3/dm/cmass1,s12.txt
saving cmass2,s12 to single_source_ClY3/dm/cmass2,s12.txt
saving s12,s12 to single_source_ClY3/dm/s12,s12.txt
saving cosmo_dict to single_source_ClY3/dm/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s12 to galaxy_sample_s12.json
CPU times: user 13.6 s, sys: 121 ms, total: 13.7 s
Wall time: 13.1 s


### List of models to compute $C(l)$ to compute **double** source bins anlaysis

0. fiducial = Planck cosmology + fiducial galaxy bias, magnification bais + 0 photo-z and multiplicative bias
1. $\omega_b h^2$ +
2. $\omega_c h^2$ +
3. $\Omega_{\rm de}$ +
4. $n_{\rm s}$ +
5. $\ln 10^{10}A_{\rm s}$ +
6. $b_{\rm 1,1}$ + 
7. $b_{\rm 1,2}$ + 
8. $b_{\rm 1,3}$ + 
9. $\alpha_{\rm mag,1}$ + 
10. $\alpha_{\rm mag,2}$ + 
11. $\alpha_{\rm mag,3}$ + 
12. $\Delta z_{\rm ph,1}$ +
12. $\Delta z_{\rm ph,2}$ +
13. $\Delta m_1$ + = easy. Just multiply $1+\Delta m_1$ to fiducial.
13. $\Delta m_2$ + = easy. Just multiply $1+\Delta m_2$ to fiducial.

In [22]:
dirname = 'double_source_ClY3'

In [23]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
pk2cl.set_cosmology_from_dict()
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=True)
pk2cl.dump_Cl_cache(f'{dirname}/fiducial', overwrite=True)

:21.839945316314697 sec
saving l to double_source_ClY3/fiducial/l.txt
saving lowz,lowz to double_source_ClY3/fiducial/lowz,lowz.txt
saving lowz,cmass1 to double_source_ClY3/fiducial/lowz,cmass1.txt
saving lowz,cmass2 to double_source_ClY3/fiducial/lowz,cmass2.txt
saving cmass1,lowz to double_source_ClY3/fiducial/cmass1,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/fiducial/cmass1,cmass1.txt
saving cmass1,cmass2 to double_source_ClY3/fiducial/cmass1,cmass2.txt
saving cmass2,lowz to double_source_ClY3/fiducial/cmass2,lowz.txt
saving cmass2,cmass1 to double_source_ClY3/fiducial/cmass2,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/fiducial/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/fiducial/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/fiducial/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/fiducial/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/fiducial/s2,s2.txt
saving cosmo_dict to double_source_ClY3/fiducial/cosmo_dict.json.
saved galaxy sample lo

In [24]:
cosmo_fiducial = pk2cl.get_cosmo_dict()
dp = 0.0001

In [25]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_b'] = cosmo_fiducial['omega_b']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_b', overwrite=True)

:13.372735023498535 sec
saving l to double_source_ClY3/omega_b/l.txt
saving lowz,lowz to double_source_ClY3/omega_b/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/omega_b/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/omega_b/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/omega_b/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/omega_b/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/omega_b/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/omega_b/s2,s2.txt
saving cosmo_dict to double_source_ClY3/omega_b/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 14 s, sys: 156 ms, total: 14.2 s
Wall time: 13.7 s


In [26]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['omega_c'] = cosmo_fiducial['omega_c']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/omega_c', overwrite=True)

:12.423795938491821 sec
saving l to double_source_ClY3/omega_c/l.txt
saving lowz,lowz to double_source_ClY3/omega_c/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/omega_c/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/omega_c/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/omega_c/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/omega_c/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/omega_c/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/omega_c/s2,s2.txt
saving cosmo_dict to double_source_ClY3/omega_c/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 116 ms, total: 13.3 s
Wall time: 12.7 s


In [27]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['Omega_de'] = cosmo_fiducial['Omega_de']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/Omega_de', overwrite=True)

:12.549093246459961 sec
saving l to double_source_ClY3/Omega_de/l.txt
saving lowz,lowz to double_source_ClY3/Omega_de/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/Omega_de/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/Omega_de/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/Omega_de/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/Omega_de/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/Omega_de/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/Omega_de/s2,s2.txt
saving cosmo_dict to double_source_ClY3/Omega_de/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.3 s, sys: 120 ms, total: 13.4 s
Wall time: 12.9 s


In [28]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['n_s'] = cosmo_fiducial['n_s']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/n_s', overwrite=True)

:12.576951026916504 sec
saving l to double_source_ClY3/n_s/l.txt
saving lowz,lowz to double_source_ClY3/n_s/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/n_s/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/n_s/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/n_s/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/n_s/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/n_s/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/n_s/s2,s2.txt
saving cosmo_dict to double_source_ClY3/n_s/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.3 s, sys: 123 ms, total: 13.4 s
Wall time: 12.9 s


In [29]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
cosmo_dict['ln10p10As'] = cosmo_fiducial['ln10p10As']*(1.0+dp)
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/ln10p10As', overwrite=True)

:12.582479000091553 sec
saving l to double_source_ClY3/ln10p10As/l.txt
saving lowz,lowz to double_source_ClY3/ln10p10As/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/ln10p10As/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/ln10p10As/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/ln10p10As/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/ln10p10As/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/ln10p10As/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/ln10p10As/s2,s2.txt
saving cosmo_dict to double_source_ClY3/ln10p10As/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.3 s, sys: 133 ms, total: 13.4 s
Wall time: 12.9 s


In [30]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78*(1.0+dp), '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1lowz', overwrite=True)

:12.396514892578125 sec
saving l to double_source_ClY3/b1lowz/l.txt
saving lowz,lowz to double_source_ClY3/b1lowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/b1lowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/b1lowz/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/b1lowz/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/b1lowz/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/b1lowz/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/b1lowz/s2,s2.txt
saving cosmo_dict to double_source_ClY3/b1lowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.1 s, sys: 121 ms, total: 13.2 s
Wall time: 12.7 s


In [31]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10*(1+dp), '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass1', overwrite=True)

:11.959874153137207 sec
saving l to double_source_ClY3/b1cmass1/l.txt
saving lowz,lowz to double_source_ClY3/b1cmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/b1cmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/b1cmass1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/b1cmass1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/b1cmass1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/b1cmass1/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/b1cmass1/s2,s2.txt
saving cosmo_dict to double_source_ClY3/b1cmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.7 s, sys: 115 ms, total: 12.8 s
Wall time: 12.3 s


In [32]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28*(1+dp), '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/b1cmass2', overwrite=True)

:11.659494876861572 sec
saving l to double_source_ClY3/b1cmass2/l.txt
saving lowz,lowz to double_source_ClY3/b1cmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/b1cmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/b1cmass2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/b1cmass2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/b1cmass2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/b1cmass2/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/b1cmass2/s2,s2.txt
saving cosmo_dict to double_source_ClY3/b1cmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.4 s, sys: 99.7 ms, total: 12.5 s
Wall time: 12 s


In [33]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259*(1.0+dp)])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamaglowz', overwrite=True)

:11.516288995742798 sec
saving l to double_source_ClY3/alphamaglowz/l.txt
saving lowz,lowz to double_source_ClY3/alphamaglowz/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/alphamaglowz/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/alphamaglowz/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/alphamaglowz/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/alphamaglowz/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/alphamaglowz/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/alphamaglowz/s2,s2.txt
saving cosmo_dict to double_source_ClY3/alphamaglowz/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.3 s, sys: 98.3 ms, total: 12.4 s
Wall time: 11.8 s


In [34]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563*(1+dp)])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass1', overwrite=True)

:12.226478099822998 sec
saving l to double_source_ClY3/alphamagcmass1/l.txt
saving lowz,lowz to double_source_ClY3/alphamagcmass1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/alphamagcmass1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/alphamagcmass1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/alphamagcmass1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/alphamagcmass1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/alphamagcmass1/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/alphamagcmass1/s2,s2.txt
saving cosmo_dict to double_source_ClY3/alphamagcmass1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.9 s, sys: 121 ms, total: 13.1 s
Wall time: 12.5 s


In [35]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729*(1+dp)])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/alphamagcmass2', overwrite=True)

:11.987351179122925 sec
saving l to double_source_ClY3/alphamagcmass2/l.txt
saving lowz,lowz to double_source_ClY3/alphamagcmass2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/alphamagcmass2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/alphamagcmass2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/alphamagcmass2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/alphamagcmass2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/alphamagcmass2/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/alphamagcmass2/s2,s2.txt
saving cosmo_dict to double_source_ClY3/alphamagcmass2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.7 s, sys: 109 ms, total: 12.8 s
Wall time: 12.3 s


In [36]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', dp, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph1', overwrite=True)

:11.994633913040161 sec
saving l to double_source_ClY3/dzph1/l.txt
saving lowz,lowz to double_source_ClY3/dzph1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/dzph1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/dzph1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/dzph1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/dzph1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/dzph1/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/dzph1/s2,s2.txt
saving cosmo_dict to double_source_ClY3/dzph1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 12.7 s, sys: 115 ms, total: 12.8 s
Wall time: 12.3 s


In [37]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', dp, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dzph2', overwrite=True)

:12.989307880401611 sec
saving l to double_source_ClY3/dzph2/l.txt
saving lowz,lowz to double_source_ClY3/dzph2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/dzph2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/dzph2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/dzph2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/dzph2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/dzph2/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/dzph2/s2,s2.txt
saving cosmo_dict to double_source_ClY3/dzph2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.6 s, sys: 160 ms, total: 13.8 s
Wall time: 13.3 s


In [38]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, dp, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':Omega_s_HSC, 'xi':Omega_s_HSC}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm1', overwrite=True)

:13.024590015411377 sec
saving l to double_source_ClY3/dm1/l.txt
saving lowz,lowz to double_source_ClY3/dm1/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/dm1/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/dm1/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/dm1/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/dm1/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/dm1/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/dm1/s2,s2.txt
saving cosmo_dict to double_source_ClY3/dm1/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13.6 s, sys: 167 ms, total: 13.8 s
Wall time: 13.3 s


In [39]:
%%time
g_l1 = hsc3x2pt.galaxy_sample_lens_class(['lowz'  , 0.15, 0.35, 1.78, '40.36 arcmin^-2', 2.259])
g_l2 = hsc3x2pt.galaxy_sample_lens_class(['cmass1', 0.47, 0.55, 2.10, '20.06 arcmin^-2', 3.563])
g_l3 = hsc3x2pt.galaxy_sample_lens_class(['cmass2', 0.55, 0.70, 2.28, '30.12 arcmin^-2', 3.729])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s1', 'sourcePzs_Y1/MLZs1.txt', 0.0, 0.0, 0.2, '4.37 arcmin^-2'])
g_s12= hsc3x2pt.galaxy_sample_source_class(['s2', 'sourcePzs_Y1/MLZs2.txt', 0.0, dp, 0.2, '4.37 arcmin^-2'])
pk2cl = hsc3x2pt.pk2cl_class(power_b1)
pk2cl.set_galaxy_sample(g_l1)
pk2cl.set_galaxy_sample(g_l2)
pk2cl.set_galaxy_sample(g_l3)
pk2cl.set_galaxy_sample(g_s12)
cosmo_dict = cosmo_fiducial.copy()
pk2cl.set_cosmology_from_dict(cosmo_dict)
pk2cl.init_pk()
pk2cl.set_Omega_s({'w':8300, 'gamma_t':140, 'xi':140}) # HSCY1

l = np.logspace(-2, 5, 1000)
with hsc3x2pt.Time():
    pk2cl.compute_all_Cl(l, compute_lens_cross=False)
pk2cl.dump_Cl_cache(f'{dirname}/dm2', overwrite=True)

:12.410616874694824 sec
saving l to double_source_ClY3/dm2/l.txt
saving lowz,lowz to double_source_ClY3/dm2/lowz,lowz.txt
saving cmass1,cmass1 to double_source_ClY3/dm2/cmass1,cmass1.txt
saving cmass2,cmass2 to double_source_ClY3/dm2/cmass2,cmass2.txt
saving lowz,s2 to double_source_ClY3/dm2/lowz,s2.txt
saving cmass1,s2 to double_source_ClY3/dm2/cmass1,s2.txt
saving cmass2,s2 to double_source_ClY3/dm2/cmass2,s2.txt
saving s2,s2 to double_source_ClY3/dm2/s2,s2.txt
saving cosmo_dict to double_source_ClY3/dm2/cosmo_dict.json.
saved galaxy sample lowz to galaxy_sample_lowz.json
saved galaxy sample cmass1 to galaxy_sample_cmass1.json
saved galaxy sample cmass2 to galaxy_sample_cmass2.json
saved galaxy sample s2 to galaxy_sample_s2.json
CPU times: user 13 s, sys: 143 ms, total: 13.1 s
Wall time: 12.8 s


In [40]:
t1 = time.time()
print(f'{t1-t0} sec')

408.02498722076416 sec
